In [2]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support import expected_conditions as EC
import os
from dotenv import load_dotenv
import csv
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import Select
from datetime import date, timedelta, datetime
from threading import Thread
import threading
from typing import List

In [3]:
load_dotenv()
edge_driver_path = os.getenv('EDGE_DRIVER_PATH')
driver = webdriver.Edge(service=Service(edge_driver_path))
driver2 = webdriver.Edge(service=Service(edge_driver_path))
driver3 = webdriver.Edge(service=Service(edge_driver_path))
lock = threading.Lock()
list_kpop_mv_url = "https://kpop.daisuki.com.br/en/mvs.html?options=101&tags=dhcmoravxpnezjsl&cols=101110000&ord=5D&fbclid=IwAR2rpByeOSp6GktqkzWLI5ipwDLLit6IE3qEhdpLemqmeBtn8C6xiLRQYo8"
detail_url = "https://kpop.daisuki.com.br/mv.php?layout=1&id="

# prepare vpn for each driver

In [5]:
driver.get(list_kpop_mv_url)

In [5]:
more_button = driver.find_element(By.XPATH, '//*[@id="morebtn"]')
driver.execute_script("arguments[0].click();", more_button)

In [6]:
list_size_cbb = driver.find_element(By.XPATH, '//*[@id="hlimit"]')
select = Select(list_size_cbb)
select.select_by_visible_text('500 items')

published_date_inp_start = driver.find_element(By.XPATH, '//*[@id="fgyear"]')
published_date_inp_end = driver.find_element(By.XPATH, '//*[@id="fgyeared"]')

start_date = date(2014, 1, 1)
end_date = date(2024, 1, 1)
list_popup_id = []
while start_date <= end_date:
    published_date_inp_start.clear()
    published_date_inp_start.send_keys(start_date.strftime('%Y-%m-%d'))

    iteration_end_date = start_date + timedelta(days=30)
    if iteration_end_date > end_date:
        iteration_end_date = end_date

    published_date_inp_end.clear()
    published_date_inp_end.send_keys(iteration_end_date.strftime('%Y-%m-%d'))

    print(f"Start date: {start_date.strftime('%Y-%m-%d')}")
    print(f"End date: {iteration_end_date.strftime('%Y-%m-%d')}")

    update_list_btn = driver.find_element(By.XPATH, '//*[@id="btngenerate"]')
    driver.execute_script("arguments[0].click();", update_list_btn)

    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'list_tiles_item')))
    list_items = driver.find_elements(By.CLASS_NAME, 'list_tiles_item')
    print(f"Number of items: {len(list_items)}")


    for item in list_items:
        id = item.get_attribute('id')
        list_popup_id.append(int(id.replace('v', '')))

    start_date += timedelta(days=30)
filename = 'popup_id_data.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for id in list_popup_id:
        writer.writerow([id])


Start date: 2014-01-01
End date: 2014-01-31
Number of items: 65
Start date: 2014-01-31
End date: 2014-03-02
Number of items: 71
Start date: 2014-03-02
End date: 2014-04-01
Number of items: 68
Start date: 2014-04-01
End date: 2014-05-01
Number of items: 43
Start date: 2014-05-01
End date: 2014-05-31
Number of items: 90
Start date: 2014-05-31
End date: 2014-06-30
Number of items: 85
Start date: 2014-06-30
End date: 2014-07-30
Number of items: 79
Start date: 2014-07-30
End date: 2014-08-29
Number of items: 104
Start date: 2014-08-29
End date: 2014-09-28
Number of items: 71
Start date: 2014-09-28
End date: 2014-10-28
Number of items: 73
Start date: 2014-10-28
End date: 2014-11-27
Number of items: 100
Start date: 2014-11-27
End date: 2014-12-27
Number of items: 62
Start date: 2014-12-27
End date: 2015-01-26
Number of items: 77
Start date: 2015-01-26
End date: 2015-02-25
Number of items: 60
Start date: 2015-02-25
End date: 2015-03-27
Number of items: 96
Start date: 2015-03-27
End date: 2015-

In [29]:
list_popup_id = []

filename = 'popup_id_data.csv'
# Read from the file
with open(filename, 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        list_popup_id.append(row[0])

In [5]:
class VideoData:
    def __init__(self, title, release_date, youtube_id, artist_name, total_organic_views, total_likes, artist_organic_views, artist_debut_date, branch_physical_sales, youtube_followers):
        self.title = title
        self.release_date = release_date
        self.youtube_id = youtube_id
        self.artist_name = artist_name
        self.organic_views = total_organic_views
        self.likes = total_likes
        self.artist_organic_views = artist_organic_views
        self.artist_debut_date = artist_debut_date
        self.branch_physical_sales = branch_physical_sales
        self.youtube_followers = youtube_followers

    def __str__(self):
        return f"title: {self.title}, release_date: {self.release_date}, youtube_id: {self.youtube_id}, artist_name: {self.artist_name}, organic_views: {self.organic_views}, likes: {self.likes}, artist_organic_views: {self.artist_organic_views}, artist_debut_date: {self.artist_debut_date}, branch_physical_sales: {self.branch_physical_sales}, youtube_followers: {self.youtube_followers}"

class ArtistData:
    def __init__(self, artist_organic_views, artist_debut_date, branch_physical_sales, youtube_followers):
        self.artist_organic_views = artist_organic_views
        self.artist_debut_date = artist_debut_date
        self.branch_physical_sales = branch_physical_sales
        self.youtube_followers = youtube_followers

In [30]:
drivers = [driver, driver2, driver3]
start_idx = 0
end_idx = 40
list_popup_id = list_popup_id[::-1]
crawl_list = list_popup_id[start_idx:end_idx]
length = len(crawl_list)
part1 = crawl_list[0:len(crawl_list)//3]
part2 = crawl_list[len(crawl_list)//3:2*len(crawl_list)//3]
part3 = crawl_list[2*len(crawl_list)//3:]

searched_artist_data = {}
process = 0


In [33]:
fieldnames = ['title', 'release_date', 'youtube_id', 'artist_name', 'organic_views', 'likes', 'artist_organic_views', 'artist_debut_date', 'branch_physical_sales', 'youtube_followers']

with open('video_data.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

In [31]:
def process_crawl_details(driver: WebDriver, ids):
    global process
    videoData_list: List[VideoData] = []
    idx = 1
    for id in ids:
        try:
            time.sleep(3)
            driver.get(detail_url + str(id))
            driver.execute_script("expandcontent('expcontent','morebtn');")

            type = driver.find_element(By.XPATH, '//li[contains(text(), "Type: ")]/b').text
            if type != 'Main':
                continue

            try:
                artist_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//li[contains(text(), "Artist: ")]/b/a'))).text
            except (NoSuchElementException, TimeoutException):
                artist_name = 'Null'

            try:
                title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h1'))).text.replace('Video Details - ', '').replace(f' - {artist_name}', '')
            except (NoSuchElementException, TimeoutException):
                title = 'Null'

            try:
                release_date = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//li[contains(text(), "Date Released: ")]/b'))).text
            except (NoSuchElementException, TimeoutException):
                release_date = 'Null'

            try:
                artist_url = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//li[contains(text(), "Artist: ")]/b/a'))).get_attribute('href')
            except (NoSuchElementException, TimeoutException):
                artist_url = 'Null'

            try:
                youtube_id = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//li[contains(text(), "Youtube ID:	")]/b/a'))).text
            except (NoSuchElementException, TimeoutException):
                youtube_id = 'Null'

            total_organic_views = 0
            total_likes = 0
            try:
                driver.find_element(By.XPATH, '//div[@class="responsive_pad_duo panel"]')
                start_date = datetime.strptime(release_date, '%Y-%m-%d')
                end_date = start_date + timedelta(days=30)
                current_date = start_date
                while current_date <= end_date:
                    organic_views = driver.find_element(By.XPATH, f'//tr[td="{current_date.strftime("%m-%d")}"]/td[3]').text
                    likes = driver.find_element(By.XPATH, f'//tr[td="{current_date.strftime("%m-%d")}"]/td[4]').text
                    if organic_views == '-':
                        organic_views = '0'
                    if likes == '-':
                        likes = '0'
                    organic_views = int(organic_views.replace('.', ''))
                    likes = int(likes.replace('.', ''))
                    # Perform addition
                    total_organic_views += organic_views
                    total_likes += likes
                    current_date += timedelta(days=1)
            except Exception as e:
                total_organic_views = 'Null'
                total_likes = 'Null'
            
            artist_organic_views = 'Null'
            artist_debut_date = 'Null'
            branch_physical_sales = 'Null'
            youtube_followers = 'Null'
            artist_data = None
            if artist_url in searched_artist_data:
                artist_data = searched_artist_data[artist_url]
            else:
                time.sleep(3)
                driver.get(artist_url)
                try:
                    artist_organic_views = driver.find_element(By.XPATH, '//td[small[contains(text(), "Organic Views")]]/b').text
                except NoSuchElementException:
                    artist_organic_views = 'Null'

                try:
                    artist_debut_date = driver.find_element(By.XPATH, '//tr[@class="dashbottom"][td[contains(text(), "Debut Day")]]/td/b').text
                except NoSuchElementException:
                    artist_debut_date = 'Null'

                try:
                    branch_physical_sales = driver.find_element(By.XPATH, '//tr[@class="dashbottom"][td[contains(text(), "Brand Physical Sales")]]/td/b').text
                except NoSuchElementException:
                    branch_physical_sales = 'Null'

                try:
                    youtube_followers = driver.find_element(By.XPATH, '//tr[@class="dashbottom"][td[contains(text(), "Youtube Followers")]]/td/b').text
                except NoSuchElementException:
                    youtube_followers = 'Null'

                artist_data = ArtistData(artist_organic_views, artist_debut_date, branch_physical_sales, youtube_followers)

                with lock:
                    searched_artist_data[artist_url] = artist_data

            videoData_list.append(VideoData(title, release_date, youtube_id, artist_name, total_organic_views, total_likes, artist_data.artist_organic_views, artist_data.artist_debut_date, artist_data.branch_physical_sales, artist_data.youtube_followers))

            with lock:
                if len(videoData_list) % 5 == 0:
                    with open('video_data.csv', 'a', newline='', encoding='utf-8') as csvfile:
                        fieldnames = ['title', 'release_date', 'youtube_id', 'artist_name', 'organic_views', 'likes', 'artist_organic_views', 'artist_debut_date', 'branch_physical_sales', 'youtube_followers']
                        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                        for video in videoData_list:
                            try:
                                writer.writerow({
                                    'title': video.title, 
                                    'release_date': video.release_date, 
                                    'youtube_id': video.youtube_id, 
                                    'artist_name': video.artist_name, 
                                    'organic_views': video.organic_views, 
                                    'likes': video.likes, 
                                    'artist_organic_views': video.artist_organic_views, 
                                    'artist_debut_date': video.artist_debut_date, 
                                    'branch_physical_sales': video.branch_physical_sales, 
                                    'youtube_followers': video.youtube_followers
                                })
                                print(f"Writing video data to CSV: {threading.get_ident()}")
                            except Exception as e:
                                print(f"Error writing video data to CSV: {e}")
                        videoData_list.clear()
                process += 1

            idx += 1
            
            print(f"thread {threading.get_ident()}, idx: {idx}, title: {title}, release_date: {release_date}, youtube_id: {youtube_id}, artist_name: {artist_name}, artist_url: {artist_url}, organic_views: {total_organic_views}, likes: {total_likes}, artist_organic_views: {artist_organic_views}, artist_debut_date: {artist_debut_date}, branch_physical_sales: {branch_physical_sales}, youtube_followers: {youtube_followers}")
        except:
            print(f"Error: {id}")
            continue

In [34]:
def print_process():
    global length
    global process
    while True:
        print(f"crawled songs: {process}")
        time.sleep(10)
print_thread = Thread(target=print_process)
print_thread.start()

thread1 = Thread(target=process_crawl_details, args=(drivers[0], part1))
thread2 = Thread(target=process_crawl_details, args=(drivers[1], part2))
thread3 = Thread(target=process_crawl_details, args=(drivers[2], part3))

thread1.start()
thread2.start()
thread3.start()

# Wait for all threads to finish
thread1.join()
thread2.join()
thread3.join()
print_thread.join()

crawled songs: 9
len video: 1
thread 32368, idx: 2, title: Love Begins With Confession, release_date: 2024-03-25, youtube_id: 3aT_hhqlBCI, artist_name: Yewon, artist_url: https://kpop.daisuki.com.br/artists/Jeong-Yewon.html, organic_views: 478, likes: 16, artist_organic_views: Null, artist_debut_date: Null, branch_physical_sales: Null, youtube_followers: Null
crawled songs: 10
len video: 1
thread 30424, idx: 2, title: To You, release_date: 2024-03-27, youtube_id: bwVj_K6rB5c, artist_name: Ryu Jihyun, artist_url: https://kpop.daisuki.com.br/artists/Ryu-Jihyun.html, organic_views: 1082, likes: 122, artist_organic_views: Null, artist_debut_date: Null, branch_physical_sales: Null, youtube_followers: Null
crawled songs: 11
len video: 2
thread 30424, idx: 3, title: Nothing Ever, release_date: 2024-03-26, youtube_id: CU4MijnavoE, artist_name: Kim Yuna (unakeii), artist_url: https://kpop.daisuki.com.br/artists/Kim-Yuna(unakeii).html, organic_views: 6280, likes: 237, artist_organic_views: Null,

In [32]:
process_crawl_details(driver, crawl_list)

len video: 1
thread 17240, idx: 2, title: Love Begins With Confession, release_date: 2024-03-25, youtube_id: 3aT_hhqlBCI, artist_name: Yewon, artist_url: https://kpop.daisuki.com.br/artists/Jeong-Yewon.html, organic_views: 478, likes: 16, artist_organic_views: 172.2K+, artist_debut_date: 2019-03-12, branch_physical_sales: Null, youtube_followers: 4.810+
len video: 2
thread 17240, idx: 3, title: To You, release_date: 2024-03-27, youtube_id: bwVj_K6rB5c, artist_name: Ryu Jihyun, artist_url: https://kpop.daisuki.com.br/artists/Ryu-Jihyun.html, organic_views: 1082, likes: 122, artist_organic_views: 346.8K+, artist_debut_date: 2015-11-18, branch_physical_sales: Null, youtube_followers: 3.020+
len video: 3
thread 17240, idx: 4, title: Nothing Ever, release_date: 2024-03-26, youtube_id: CU4MijnavoE, artist_name: Kim Yuna (unakeii), artist_url: https://kpop.daisuki.com.br/artists/Kim-Yuna(unakeii).html, organic_views: 6280, likes: 237, artist_organic_views: 143.2K+, artist_debut_date: 2018-03-